# OLID datasets

## import libraries

In [1]:
import pandas as pd
import string
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split


## load the datasets

In [3]:
train_data=pd.read_csv('./olid-training-v1.0.tsv', delimiter='\t', encoding='utf-8')

train_tweets = train_data[['tweet']] #Extract tweets
train_task_a_labels= train_data[['subtask_a']] #Extract subtsak_a labels
train_task_b_labels= train_data[['subtask_b']] #Extract subtsak_b labels
train_task_c_labels= train_data[['subtask_c']] #Extract subtsak_c labels

train_task_a_labels.columns.values[0] = 'class_a' #Rename class attribute
train_task_b_labels.columns.values[0] = 'class_b' #Rename class attribute
train_task_c_labels.columns.values[0] = 'class_c' #Rename class attribute

print(train_data)

          id                                              tweet subtask_a  \
0      86426  @USER She should ask a few native Americans wh...       OFF   
1      90194  @USER @USER Go home you’re drunk!!! @USER #MAG...       OFF   
2      16820  Amazon is investigating Chinese employees who ...       NOT   
3      62688  @USER Someone should'veTaken" this piece of sh...       OFF   
4      43605  @USER @USER Obama wanted liberals &amp; illega...       NOT   
...      ...                                                ...       ...   
13235  95338  @USER Sometimes I get strong vibes from people...       OFF   
13236  67210  Benidorm ✅  Creamfields ✅  Maga ✅   Not too sh...       NOT   
13237  82921  @USER And why report this garbage.  We don't g...       OFF   
13238  27429                                        @USER Pussy       OFF   
13239  46552  #Spanishrevenge vs. #justice #HumanRights and ...       NOT   

      subtask_b subtask_c  
0           UNT       NaN  
1           TIN    

In [4]:
train_data

,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN
...,...,...,...,...,...
13235,95338,@USER Sometimes I get strong vibes from people...,OFF,TIN,IND
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT,NaN,NaN
13237,82921,@USER And why report this garbage. We don't g...,OFF,TIN,OTH
13238,27429,@USER Pussy,OFF,UNT,NaN


In [5]:
train_data['subtask_a'].value_counts()

NOT    8840
OFF    4400
Name: subtask_a, dtype: int64

## NLP

In [6]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 7.9 MB/s eta 0:00:00


In [7]:
import emoji
import re

In [8]:
def replace_multiple_user_mentions(text):
    """
    Replaces multiple '@USER' mentions in a tweet with a single '@USERS'.
    """
    user_mentions = len(re.findall('@USER', text))
    if user_mentions > 1:
        text = re.sub('@USER', '@USERS', text)
    return text


In [9]:

def emoji_to_text(text):
    return emoji.demojize(text, delimiters=("", ""))

In [10]:
def clean_tweets(df):
    punctuations = string.punctuation

    df.loc[:, 'tweet'] = df.tweet.str.replace('@USER', '') #Remove mentions (@USER)
    df.loc[:, 'tweet'] = df.tweet.str.replace('URL', '') #Remove URLs
    df.loc[:, 'tweet'] = df.tweet.str.replace('&amp', 'and') #Replace ampersand (&) with and
    df.loc[:, 'tweet'] = df.tweet.str.replace('&lt','') #Remove &lt
    df.loc[:, 'tweet'] = df.tweet.str.replace('&gt','') #Remove &gt
    df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
    df.loc[:, 'tweet'] = df.tweet.str.lower() #Lowercase

    #Remove punctuations
    for punctuation in punctuations:
        df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')

        df['tweet'] = df['tweet'].apply(
        lambda x: x.encode('ascii', 'ignore').decode('ascii')
)

    # Transfer emojis to segmentations
    df.loc[:, 'tweet'] = df['tweet'].apply(emoji.demojize)
    #Remove emojis
    #df.loc[:, 'tweet'] = df.tweet.str.strip() #Trim leading and trailing whitespaces
    # Replace multiple user mentions
    df.loc[:, 'tweet'] = df['tweet'].apply(replace_multiple_user_mentions)

    # Substitute out-of-vocabulary (OOV) words
    df.loc[:, 'tweet'] = df['tweet'].apply(lambda x: re.sub(r'http\S+|www\S+', 'http', x))


In [11]:
clean_tweets(train_tweets)

<ipython-input-10-58d0d99cd0a4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'tweet'] = df.tweet.str.replace('@USER', '') #Remove mentions (@USER)
<ipython-input-10-58d0d99cd0a4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'tweet'] = df.tweet.str.replace('URL', '') #Remove URLs
<ipython-input-10-58d0d99cd0a4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [12]:
train_task_a_data = train_tweets.join(train_task_a_labels)

#train_task_b_data = train_tweets.join(train_task_b_labels)
#train_task_b_data = train_task_b_data.dropna() #Drop records with missing values

#train_task_c_data = train_tweets.join(train_task_c_labels)
#train_task_c_data = train_task_c_data.dropna() #Drop records with missing values

#Apply quotes to cleaned tweets
train_task_a_data.update(train_task_a_data[['tweet']].applymap('\'{}\''.format))
#train_task_b_data.update(train_task_b_data[['tweet']].applymap('\'{}\''.format))
#train_task_c_data.update(train_task_c_data[['tweet']].applymap('\'{}\''.format))

In [13]:
train_task_a_data.head()

,tweet,class_a
0,' she should ask a few native americans what t...,OFF
1,' go home youre drunk maga trump ',OFF
2,'amazon is investigating chinese employees who...,NOT
3,' someone shouldvetaken this piece of shit to ...,OFF
4,' obama wanted liberals and illegals to move ...,NOT


In [15]:
#Read tweets from test sets
test_tweet_a=pd.read_csv('./testset-levela.tsv', delimiter='\t', encoding='utf-8')
#test_tweet_b=pd.read_csv('../raw_data/testset-levelb.tsv', delimiter='\t', encoding='utf-8')
#test_tweet_c=pd.read_csv('../raw_data/testset-levelc.tsv', delimiter='\t', encoding='utf-8')

#Read tweet labels
test_label_a=pd.read_csv('./labels-levela.csv', encoding='utf-8',
                         index_col=False, names=['id', 'class_a'])
#test_label_b=pd.read_csv('../raw_data/labels-levelb.csv', encoding='utf-8',
                       #  index_col=False, names=['id', 'class_b'])
#test_label_c=pd.read_csv('../raw_data/labels-levelc.csv', encoding='utf-8',
                       #  index_col=False, names=['id', 'class_c'])

#Merge tweets with labels by id
test_tweet_a = test_tweet_a.merge(test_label_a, on='id')
#test_tweet_b = test_tweet_b.merge(test_label_b, on='id')
#test_tweet_c = test_tweet_c.merge(test_label_c, on='id')

#Drop id column
test_tweet_a = test_tweet_a.drop(columns='id')
#test_tweet_b = test_tweet_b.drop(columns='id')
#test_tweet_c = test_tweet_c.drop(columns='id')

#Clean tweets in test sets
clean_tweets(test_tweet_a)
#clean_tweets(test_tweet_b)
#clean_tweets(test_tweet_c)

#Apply quotes to cleaned tweets
test_tweet_a.update(test_tweet_a[['tweet']].applymap('\'{}\''.format))
#test_tweet_b.update(test_tweet_b[['tweet']].applymap('\'{}\''.format))
#test_tweet_c.update(test_tweet_c[['tweet']].applymap('\'{}\''.format))

test_tweet_a.head()

<ipython-input-10-58d0d99cd0a4>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
<ipython-input-10-58d0d99cd0a4>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')


,tweet,class_a
0,'whoisq wherestheserver dumpnike declasfisa de...,OFF
1,'constitutionday is revered by conservatives h...,NOT
2,'foxnews nra maga potus trump ndamendment rnc ...,NOT
3,'watching boomer getting the news that she is ...,NOT
4,'nopasaran unity demo to oppose the farright i...,OFF


# Task A

In [16]:
# Creating labelEncoder
encoder = LabelEncoder()
# Converting string labels into numbers.
train_task_a_data["class_a_code"] = encoder.fit_transform(train_task_a_data["class_a"])
test_tweet_a["class_a_code"] = encoder.fit_transform(test_tweet_a["class_a"])

In [17]:
#Create tuple pair for class and class code
train_task_a_data['class-tuple'] = train_task_a_data[['class_a', 'class_a_code']].apply(tuple, axis=1)
class_a = train_task_a_data['class-tuple'].unique()
class_a

array([('OFF', 1), ('NOT', 0)], dtype=object)

In [18]:
# Label the Data

train_tweets = train_task_a_data['tweet']
train_labels = train_task_a_data['class_a_code']

X_test = test_tweet_a['tweet']
y_test = test_tweet_a['class_a_code']


print("Shape of train_tweet is {} and shape of train_label is {}".format(train_tweets.shape, train_labels.shape))
print("Shape of test_tweet is {} and shape of test_label is {}".format(X_test.shape, y_test.shape))

Shape of train_tweet is (13240,) and shape of train_label is (13240,)
Shape of test_tweet is (860,) and shape of test_label is (860,)


In [19]:
X_train,X_val,y_train,y_val = train_test_split(train_tweets,train_labels,test_size=0.30,random_state=42)


print("Shape of test_tweets is {} and shape of test_labels is {}".format(X_train.shape, y_train.shape))

Shape of test_tweets is (9268,) and shape of test_labels is (9268,)


In [20]:
X_train

2804               '  truly anything to avoid gun control'
13084    ' its not wrongfully characterizing its the fu...
2211     '  they need to add more gun control laws and ...
11154    'boom  maybe that book deal isnt such a good i...
10620    ' unfirtunately i have seen hispanics justifyi...
                               ...                        
11964    ' isnt there a statute of limitations on sexua...
5191             '  he is probably dying due to dog fumes'
5390                                  '  you are so right'
860           ' sams act is working i swear she is acting'
7270                                     '  chuckle heads'
Name: tweet, Length: 9268, dtype: object

In [21]:
y_train

2804     0
13084    1
2211     0
11154    0
10620    1
        ..
11964    0
5191     0
5390     0
860      0
7270     0
Name: class_a_code, Length: 9268, dtype: int64

In [22]:
test_tweet_a

,tweet,class_a,class_a_code
0,'whoisq wherestheserver dumpnike declasfisa de...,OFF,1
1,'constitutionday is revered by conservatives h...,NOT,0
2,'foxnews nra maga potus trump ndamendment rnc ...,NOT,0
3,'watching boomer getting the news that she is ...,NOT,0
4,'nopasaran unity demo to oppose the farright i...,OFF,1
...,...,...,...
855,'despicabledems lie again about rifles dem dis...,OFF,1
856,'meetthespeakers will present in our event o...,NOT,0
857,' people just unfollowed me for talking about ...,OFF,1
858,'wednesdaywisdom antifa calls the right fascis...,NOT,0


In [23]:
X_test

0      'whoisq wherestheserver dumpnike declasfisa de...
1      'constitutionday is revered by conservatives h...
2      'foxnews nra maga potus trump ndamendment rnc ...
3      'watching boomer getting the news that she is ...
4      'nopasaran unity demo to oppose the farright i...
                             ...                        
855    'despicabledems lie again about rifles dem dis...
856    'meetthespeakers   will present in our event o...
857    ' people just unfollowed me for talking about ...
858    'wednesdaywisdom antifa calls the right fascis...
859             'kavanaugh typical liberals  democrats '
Name: tweet, Length: 860, dtype: object

In [24]:
y_test

0      1
1      0
2      0
3      0
4      1
      ..
855    1
856    0
857    1
858    0
859    0
Name: class_a_code, Length: 860, dtype: int64

In [25]:
random_0_rows = test_tweet_a[test_tweet_a['class_a_code'] == 0].sample(n=100, random_state=42)

# Select random 50 rows with class_a_code equal to 1
random_1_rows = test_tweet_a[test_tweet_a['class_a_code'] == 1].sample(n=50, random_state=42)

# Concatenate the two DataFrames
test_tweet_a_150 = pd.concat([random_0_rows, random_1_rows])

In [26]:
test_tweet_a_150= test_tweet_a_150.sample(frac=1, random_state=42).reset_index(drop=True)

In [27]:
test_tweet_a_150

,tweet,class_a,class_a_code
0,'aurora il kanecty please keep looking for and...,NOT,0
1,' same here i cant hear what she is saying',NOT,0
2,'geoffreyowens niggas dont really understandle...,OFF,1
3,'trump may be many things but he is entertaini...,NOT,0
4,'kamalaharris may run for president in but wi...,NOT,0
...,...,...,...
145,'conservatives you cannot ride this one outvot...,NOT,0
146,' beats the hell nj out of me lol dude alre...,OFF,1
147,'watching boomer getting the news that she is ...,NOT,0
148,'brexit is dead not because the other eu state...,NOT,0


In [28]:
X_test_150 = test_tweet_a_150['tweet']
y_test_150 = test_tweet_a_150['class_a_code']

In [29]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.6 MB/s eta 0:00:00


In [30]:
from transformers import BertTokenizerFast, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features

import tensorflow as tf

# Use the same tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Encode Labels
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)
val_labels = encoder.transform(y_val)
test_labels = encoder.transform(y_test)

# Tokenize and align labels with the tokenized inputs
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(list(X_val), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=256)



In [31]:
y_test

0      1
1      0
2      0
3      0
4      1
      ..
855    1
856    0
857    1
858    0
859    0
Name: class_a_code, Length: 860, dtype: int64

In [32]:
test_labels

array([1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [33]:
# Convert to TensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(1000).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(16)

In [34]:

# Initialize the BERT model (make sure the number of labels match your dataset)
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(class_a))

# Choose an optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-6)

# Choose a loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
from tensorflow.keras.callbacks import EarlyStopping


# Define early stopping
early_stopping = EarlyStopping(
    monitor='val_loss', # usually val_loss or val_acc
    mode='min', # the direction that you want to monitor (min for loss, max for accuracy)
    patience=3, # number of epochs to wait before stopping the training
    verbose=1 # verbosity mode
)



In [36]:
# Train the model
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,
    callbacks=[early_stopping] # add early stopping to the callbacks
)

# Evaluate the model
results = model.evaluate(test_dataset)

# Get the model predictions
predictions = tf.argmax(model.predict(test_dataset)[0], axis=-1)


Epoch 1/15
580/580 [==============================] - 291s 404ms/step - loss: 0.5492 - accuracy: 0.7213 - val_loss: 0.4556 - val_accuracy: 0.7873
Epoch 2/15
580/580 [==============================] - 195s 337ms/step - loss: 0.4297 - accuracy: 0.8095 - val_loss: 0.4399 - val_accuracy: 0.8019
Epoch 3/15
580/580 [==============================] - 193s 333ms/step - loss: 0.3851 - accuracy: 0.8328 - val_loss: 0.4375 - val_accuracy: 0.8089
Epoch 4/15
580/580 [==============================] - 209s 361ms/step - loss: 0.3508 - accuracy: 0.8519 - val_loss: 0.4590 - val_accuracy: 0.8056
Epoch 5/15
580/580 [==============================] - 191s 329ms/step - loss: 0.3120 - accuracy: 0.8693 - val_loss: 0.4702 - val_accuracy: 0.8084
Epoch 6/15
580/580 [==============================] - 191s 330ms/step - loss: 0.2713 - accuracy: 0.8942 - val_loss: 0.5217 - val_accuracy: 0.7878
Epoch 6: early stopping
54/54 [==============================] - 9s 89ms/step


In [37]:
from sklearn.metrics import classification_report


# Decode the label numbers back into original labels
y_test_inverse = encoder.inverse_transform(y_test)
predictions_inverse = encoder.inverse_transform(predictions.numpy())

# Generate the classification report
report = classification_report(y_test_inverse, predictions_inverse,
                               target_names=encoder.classes_.astype(str))
print(report)


              precision    recall  f1-score   support

           0       0.90      0.83      0.86       620
           1       0.63      0.77      0.69       240

    accuracy                           0.81       860
   macro avg       0.77      0.80      0.78       860
weighted avg       0.83      0.81      0.82       860



# test dataset with 150 rows

In [318]:
random_0_rows = test_tweet_a[test_tweet_a['class_a_code'] == 0].sample(n=100, random_state=42)

# Select random 50 rows with class_a_code equal to 1
random_1_rows = test_tweet_a[test_tweet_a['class_a_code'] == 1].sample(n=50, random_state=42)

# Concatenate the two DataFrames
test_tweet_a_150 = pd.concat([random_0_rows, random_1_rows])

In [319]:
test_tweet_a_150= test_tweet_a_150.sample(frac=1, random_state=42).reset_index(drop=True)

In [320]:
X_test_150 = test_tweet_a_150['tweet']
y_test_150 = test_tweet_a_150['class_a_code']

In [321]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_150 = encoder.transform(y_test_150)
test_encodings_150 = tokenizer(list(X_test_150), truncation=True, padding=True, max_length=256)


In [322]:
test_dataset_150 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_150),
    test_labels_150
)).batch(16)

In [323]:
predictions_150 = tf.argmax(model.predict(test_dataset_150)[0], axis=-1)

10/10 [==============================] - 1s 82ms/step


In [324]:
y_test_inverse_150 = encoder.inverse_transform(y_test_150)
predictions_inverse_150 = encoder.inverse_transform(predictions_150.numpy())

# Generate the classification report
report_150 = classification_report(y_test_inverse_150, predictions_inverse_150,
                               target_names=encoder.classes_.astype(str))
print(report_150)

              precision    recall  f1-score   support

           0       0.93      0.81      0.87       100
           1       0.70      0.88      0.78        50

    accuracy                           0.83       150
   macro avg       0.81      0.84      0.82       150
weighted avg       0.85      0.83      0.84       150



In [304]:
test_tweet_off = test_tweet_a_new[test_tweet_a_new['class_a']=='OFF']

In [307]:
test_tweet_off.head()

,tweet,class_a,class_a_code
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF,1
4,#NoPasaran: Unity demo to oppose the far-right...,OFF,1
5,. . . What the fuck did he do this time?,OFF,1
7,@USER Do you get the feeling he is kissing @US...,OFF,1
14,@USER Nigga ware da hits at,OFF,1


In [306]:
test_tweet_off['class_a_code'] =1

<ipython-input-306-3a0f14904e30>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off['class_a_code'] =1


In [308]:
test_tweet_off_00 = test_tweet_off.loc[index_new2]

In [309]:
test_tweet_off_00.shape

(50, 3)

In [310]:
test_tweet_off_00.head()

,tweet,class_a,class_a_code
308,@USER @USER @USER Okay and? Anybody can go to ...,OFF,1
216,And🤷🏽‍♂️ females b on the same shit URL,OFF,1
732,#LiberalHypocrisy #TacoBell When Liberals ask ...,OFF,1
376,7 fucking years. #MyTwitterAnniversary URL,OFF,1
382,#ChristineBlaseyFord is only being harassed by...,OFF,1


In [311]:
test_tweet_a_150_00 = pd.concat([random_0_rows, test_tweet_off_00])

In [312]:
test_tweet_a_150_00= test_tweet_a_150_00.sample(frac=1, random_state=42).reset_index(drop=True)

In [313]:
X_test_150_00 = test_tweet_a_150_00['tweet']
y_test_150_00 = test_tweet_a_150_00['class_a_code']

In [314]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_150_00 = encoder.transform(y_test_150_00)
test_encodings_150_00 = tokenizer(list(X_test_150_00), truncation=True, padding=True, max_length=256)


In [315]:
test_dataset_150_00 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_150_00),
    test_labels_150_00
)).batch(16)

In [316]:
predictions_150_00 = tf.argmax(model.predict(test_dataset_150_00)[0], axis=-1)

10/10 [==============================] - 1s 113ms/step


In [317]:
y_test_inverse_150_00 = encoder.inverse_transform(y_test_150_00)
predictions_inverse_150_00 = encoder.inverse_transform(predictions_150_00.numpy())

# Generate the classification report
report_150_00 = classification_report(y_test_inverse_150_00, predictions_inverse_150_00,
                               target_names=encoder.classes_.astype(str))
print(report_150_00)

              precision    recall  f1-score   support

           0       0.82      0.81      0.81       100
           1       0.63      0.64      0.63        50

    accuracy                           0.75       150
   macro avg       0.72      0.73      0.72       150
weighted avg       0.75      0.75      0.75       150



In [ ]:
model.save('my_model_agrresive')

2023-07-22 13:53:52.370386: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: my_model_agrresive/assets


INFO:tensorflow:Assets written to: my_model_agrresive/assets


In [332]:
test_tweet_a_new=pd.read_csv('./testset-levela.tsv', delimiter='\t', encoding='utf-8')
test_label_a_new=pd.read_csv('./labels-levela.csv', encoding='utf-8',
                         index_col=False, names=['id', 'class_a'])

test_tweet_a_new = test_tweet_a_new.merge(test_label_a_new, on='id')

test_tweet_a_new = test_tweet_a_new.drop(columns='id')



In [45]:
test_tweet_a_new

,tweet,class_a
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF
1,"#ConstitutionDay is revered by Conservatives, ...",NOT
2,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT
3,#Watching #Boomer getting the news that she is...,NOT
4,#NoPasaran: Unity demo to oppose the far-right...,OFF
...,...,...
855,#DespicableDems lie again about rifles. Dem Di...,OFF
856,#MeetTheSpeakers 🙌 @USER will present in our e...,NOT
857,3 people just unfollowed me for talking about ...,OFF
858,#WednesdayWisdom Antifa calls the right fascis...,NOT


In [333]:
test_tweet_off = test_tweet_a_new[test_tweet_a_new['class_a']=='OFF']

In [47]:
test_tweet_off

,tweet,class_a
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF
4,#NoPasaran: Unity demo to oppose the far-right...,OFF
5,. . . What the fuck did he do this time?,OFF
7,@USER Do you get the feeling he is kissing @US...,OFF
14,@USER Nigga ware da hits at,OFF
...,...,...
842,"#Antifa are mentally unstable cowards, pretend...",OFF
847,@USER @USER And Browning looked like dog shit ...,OFF
853,All two of them taste like ass. URL,OFF
855,#DespicableDems lie again about rifles. Dem Di...,OFF


In [48]:
test_tweet_off['tweet'][0]

'#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon \u2066@USER URL'

In [49]:
test_tweet_off.index

Int64Index([  0,   4,   5,   7,  14,  15,  27,  29,  30,  38,
            ...
            826, 827, 833, 834, 835, 842, 847, 853, 855, 857],
           dtype='int64', length=240)

In [56]:
test_tweet_off.index.value_counts()

0      1
4      1
507    1
509    1
511    1
      ..
277    1
279    1
281    1
282    1
857    1
Length: 240, dtype: int64

In [50]:
index_list = list(test_tweet_off.index)
print(index_list)

[0, 4, 5, 7, 14, 15, 27, 29, 30, 38, 39, 40, 43, 47, 48, 51, 59, 61, 63, 64, 70, 72, 77, 80, 81, 85, 87, 88, 98, 99, 100, 107, 108, 110, 115, 129, 130, 135, 136, 138, 140, 142, 143, 147, 152, 153, 154, 156, 160, 164, 168, 173, 175, 177, 180, 182, 185, 186, 189, 196, 201, 202, 203, 207, 212, 215, 216, 221, 222, 228, 229, 233, 241, 243, 247, 249, 258, 259, 260, 262, 264, 265, 274, 275, 277, 279, 281, 282, 297, 307, 308, 309, 310, 312, 313, 314, 315, 323, 324, 326, 331, 333, 334, 335, 339, 347, 351, 352, 357, 362, 369, 370, 371, 372, 376, 377, 382, 389, 400, 403, 406, 408, 413, 414, 420, 422, 426, 428, 429, 432, 433, 437, 439, 443, 445, 447, 455, 460, 462, 468, 470, 475, 478, 480, 483, 488, 491, 494, 495, 502, 503, 505, 507, 509, 511, 512, 515, 526, 531, 533, 549, 554, 555, 559, 566, 567, 569, 572, 579, 584, 587, 591, 592, 594, 601, 609, 613, 614, 615, 617, 629, 632, 634, 637, 638, 639, 640, 649, 657, 663, 665, 667, 676, 679, 681, 690, 691, 698, 701, 702, 703, 711, 730, 732, 736, 737, 740

In [229]:
import random

In [ ]:

random_indexes = random.sample(index_list, 100)

In [ ]:
print(random_indexes)

[810, 334, 313, 475, 203, 782, 737, 282, 762, 420, 160, 826, 736, 533, 483, 755, 629, 835, 732, 347, 601, 432, 730, 326, 164, 422, 85, 249, 98, 413, 100, 377, 531, 129, 480, 308, 265, 173, 447, 443, 212, 408, 749, 478, 147, 153, 433, 613, 260, 617, 312, 216, 63, 445, 775, 572, 307, 505, 437, 800, 180, 51, 470, 30, 554, 138, 228, 791, 495, 14, 241, 681, 277, 333, 207, 426, 156, 842, 140, 667, 215, 185, 247, 676, 765, 526, 591, 491, 323, 406, 638, 751, 47, 264, 59, 460, 7, 814, 833, 258]


In [ ]:
print(random_indexes[:50])

[774, 778, 698, 488, 260, 38, 39, 594, 282, 241, 741, 443, 711, 676, 640, 331, 362, 107, 61, 460, 857, 403, 130, 786, 408, 559, 764, 281, 667, 115, 63, 853, 690, 262, 579, 168, 847, 462, 85, 40, 614, 736, 632, 352, 216, 30, 196, 313, 422, 447]


In [57]:
test_tweet_off_new = test_tweet_off

In [58]:
test_tweet_off_new

,tweet,class_a
0,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF
4,#NoPasaran: Unity demo to oppose the far-right...,OFF
5,. . . What the fuck did he do this time?,OFF
7,@USER Do you get the feeling he is kissing @US...,OFF
14,@USER Nigga ware da hits at,OFF
...,...,...
842,"#Antifa are mentally unstable cowards, pretend...",OFF
847,@USER @USER And Browning looked like dog shit ...,OFF
853,All two of them taste like ass. URL,OFF
855,#DespicableDems lie again about rifles. Dem Di...,OFF


In [271]:
test_tweet_off_new0 = test_tweet_off

In [272]:
test_tweet_off_new0.shape

(240, 2)

In [273]:
test_tweet_off_new0_0 = test_tweet_off_new0.loc[index_new2]


In [274]:

merged_df_00 = pd.concat([test_tweet_not_new, test_tweet_off_new0_0], ignore_index=True)

# Shuffle the index
test_00_df = merged_df_00.sample(frac=1,random_state=42).reset_index(drop=True)

In [275]:
clean_tweets(test_00_df)

<ipython-input-10-58d0d99cd0a4>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
<ipython-input-10-58d0d99cd0a4>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')


In [276]:
test_00_df.update(test_00_df[['tweet']].applymap('\'{}\''.format))

test_00_df.head()

,tweet,class_a
0,'gmm roastmycat hi this is fiona she is abou...,NOT
1,' fakenewsmedia is the enemyofthepeople they ...,NOT
2,'can you believe it a hundred rounds goon you ...,OFF
3,'alert a win for the progressive conservative...,NOT
4,'killshot easily shows ya why he is a a rap go...,NOT


In [277]:
test_00_df["class_a_code"] = encoder.fit_transform(test_00_df["class_a"])

In [278]:
X_test_00 = test_00_df['tweet']
y_test_00 = test_00_df['class_a_code']

In [279]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_00 = encoder.transform(y_test_00)

In [280]:
test_encodings_00 = tokenizer(list(X_test_00), truncation=True, padding=True, max_length=256)

In [281]:
test_dataset_00 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_00),
    test_labels_00
)).batch(16)

In [282]:
# Get the model predictions
predictions_00 = tf.argmax(model.predict(test_dataset_00)[0], axis=-1)


10/10 [==============================] - 1s 110ms/step


In [283]:
y_test_inverse_00 = encoder.inverse_transform(y_test_00)
predictions_inverse_00 = encoder.inverse_transform(predictions_00.numpy())

# Generate the classification report
report_00 = classification_report(y_test_inverse_00, predictions_inverse_00,
                               target_names=encoder.classes_.astype(str))
print(report_00)

              precision    recall  f1-score   support

           0       0.69      0.85      0.76       100
           1       0.44      0.24      0.31        50

    accuracy                           0.65       150
   macro avg       0.57      0.54      0.54       150
weighted avg       0.61      0.65      0.61       150



In [240]:
test_tweet_off_new['tweet'][216]

'Both men and women appear to face similar circumstances, which is worth noting. The URL could reflect an experience common to both genders.'

In [209]:
index_new = [27,370,334,702,857,243,460,827,326,30,221,617,173,85,371,567,428,740,439,408,420,638,663,377,775,502,468,177,180,422,262,335,774,555,657,201,222,511,549,509,587,771,437,800,108,207,609,156,0,857]

# ChatGPT 3.5 with rephrasing less aggressive

In [61]:
test_tweet_off_new['tweet'][27] ="Let's consider a scenario where DeLauro is put in a police lineup. Some people might point her out due to certain negative perceptions. This situation raises questions about the voters' choices or the possibility of voter fraud. It's essential to remember that everyone's right to vote should be respected, regardless of their gender or appearance. \u2066@USER @USER"

<ipython-input-61-934524491a2a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][27] ="Let's consider a scenario where DeLauro is put in a police lineup. Some people might point her out due to certain negative perceptions. This situation raises questions about the voters' choices or the possibility of voter fraud. It's essential to remember that everyone's right to vote should be respected, regardless of their gender or appearance. \u2066@USER @USER"


In [62]:
test_tweet_off_new['tweet'][370] ="Recent events involving #Beeto in #Texas serve as a demonstration of how not to run a successful election campaign. #MAGA \u2066@USER #rkba @USER #DVC UR"

<ipython-input-62-02d0ca3386c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][370] ="Recent events involving #Beeto in #Texas serve as a demonstration of how not to run a successful election campaign. #MAGA \u2066@USER #rkba @USER #DVC UR"


In [63]:
test_tweet_off_new['tweet'][334] ="#Conservative Individuals can learn valuable lessons from the gaming community. URL #Conservatives #TriggerWarningRadio"

<ipython-input-63-074a3cba6ec7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][334] ="#Conservative Individuals can learn valuable lessons from the gaming community. URL #Conservatives #TriggerWarningRadio"


In [64]:
test_tweet_off_new['tweet'][702] = "I'd like to share my opinion, even though #NoOneAskMe. Regarding the ongoing beef between #CardiB and @USER, it seems like #Offset is involved to some extent. Cardi might be trusting him, considering their relationship. However, it's important for #offset to handle the situation calmly and thoughtfully."

<ipython-input-64-48fef1184163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][702] = "I'd like to share my opinion, even though #NoOneAskMe. Regarding the ongoing beef between #CardiB and @USER, it seems like #Offset is involved to some extent. Cardi might be trusting him, considering their relationship. However, it's important for #offset to handle the situation calmly and thoughtfully."


In [65]:
test_tweet_off_new['tweet'][857]="Three people recently unfollowed me after discussing Merlin. Apologies, everyone! I'm still obsessed with Covinsky, but I'm eagerly waiting for a possible sequel announcement for PS I Still Love You. I've run out of witty and funny tweets about To All the Boys I've Loved Before; it's been quite exhausting."

<ipython-input-65-776c00a1903c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][857]="Three people recently unfollowed me after discussing Merlin. Apologies, everyone! I'm still obsessed with Covinsky, but I'm eagerly waiting for a possible sequel announcement for PS I Still Love You. I've run out of witty and funny tweets about To All the Boys I've Loved Before; it's been quite exhausting."


In [66]:
test_tweet_off_new['tweet'][243] = "#Trump's actions have raised concerns about possible obstruction of justice, as some see them as significant evidence. Critics view him as a controversial figure in this regard."

<ipython-input-66-2c95f3565eb3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][243] = "#Trump's actions have raised concerns about possible obstruction of justice, as some see them as significant evidence. Critics view him as a controversial figure in this regard."


In [67]:
test_tweet_off_new['tweet'][460] = "#Feinstein, it's important to be honest and truthful in your statements. Let's strive to maintain integrity and avoid spreading misinformation. URL"

<ipython-input-67-f8f94c069d23>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][460] = "#Feinstein, it's important to be honest and truthful in your statements. Let's strive to maintain integrity and avoid spreading misinformation. URL"


In [68]:
test_tweet_off_new['tweet'][827] = "#MAGA Can you please share one positive action that #Trump has taken during his time as #president? I'm open to hearing different perspectives. Let's discuss his accomplishments and record objectively."

<ipython-input-68-0df2d76f0c24>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][827] = "#MAGA Can you please share one positive action that #Trump has taken during his time as #president? I'm open to hearing different perspectives. Let's discuss his accomplishments and record objectively."


In [69]:
test_tweet_off_new['tweet'][326] = "#FamiliarWife JH-Oh my goodness, she's quite unpredictable. WJ-Hold on!...yes, I find assistant manager Cha interesting. BH-You two seem compatible. VO-Any issues between you? Just sit here. JH-I advise against it! WJ-Let's live life on our terms. Let's give it a try. News-There's a collision on the highway causing traffic. JH-Woo-jin~ URL"

<ipython-input-69-aa92a92473e0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][326] = "#FamiliarWife JH-Oh my goodness, she's quite unpredictable. WJ-Hold on!...yes, I find assistant manager Cha interesting. BH-You two seem compatible. VO-Any issues between you? Just sit here. JH-I advise against it! WJ-Let's live life on our terms. Let's give it a try. News-There's a collision on the highway causing traffic. JH-Woo-jin~ URL"


In [70]:
test_tweet_off_new['tweet'][30] = "#ChristineBlaseyFord is the individual who accused #Kavanaugh. It seems like this accusation is a pattern that #Liberals tend to follow each time. Let's focus on discussing the #ConfirmJudgeKavanaugh process. URL"


<ipython-input-70-73aa630e5efb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][30] = "#ChristineBlaseyFord is the individual who accused #Kavanaugh. It seems like this accusation is a pattern that #Liberals tend to follow each time. Let's focus on discussing the #ConfirmJudgeKavanaugh process. URL"


In [71]:
test_tweet_off_new['tweet'][221]="#DeclassifyDocuments It's intriguing to observe different perspectives, like Maria's, during discussions. #Outnumbered"

<ipython-input-71-36e5f963907f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][221]="#DeclassifyDocuments It's intriguing to observe different perspectives, like Maria's, during discussions. #Outnumbered"


In [72]:
test_tweet_off_new['tweet'][617] = "#LiberalismIsAMentalDisorder It's essential to recognize differing viewpoints. #Hypocrites #DemocratsHateAmerica #DemocratsAreDangerous It's important to engage in constructive dialogue and find common ground. URL"

<ipython-input-72-54d016367282>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][617] = "#LiberalismIsAMentalDisorder It's essential to recognize differing viewpoints. #Hypocrites #DemocratsHateAmerica #DemocratsAreDangerous It's important to engage in constructive dialogue and find common ground. URL"


In [73]:
test_tweet_off_new['tweet'][173] ="#Elitist #Actors #NBC #Liberals at the #Emmy Awards showcased a self-promotion event with some #MeToo references. It's worth noting that this was a planned and staged occasion, which might align with their #Democratic and #Socialist beliefs. Let's remember the importance of respectful dialogue and understanding different perspectives. #WALKAWAY URL"

<ipython-input-73-d7419fe3bb7f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][173] ="#Elitist #Actors #NBC #Liberals at the #Emmy Awards showcased a self-promotion event with some #MeToo references. It's worth noting that this was a planned and staged occasion, which might align with their #Democratic and #Socialist beliefs. Let's remember the importance of respectful dialogue and understanding different perspectives. #WALKAWAY URL"


In [74]:
test_tweet_off_new['tweet'][85] = "#Liberals can be seen differently by some, but let's remember the principle of INNOCENT UNTIL PROVEN GUILTY. It's important to examine both parties if there are concerns about individuals' conduct. Let's strive to have respectful discussions and avoid harsh language. #KavanaughIsInnocent #WalkAwayFromDemocrats. Let's focus on honesty and avoid extreme accusations. URL"

<ipython-input-74-ca5b2942eb38>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][85] = "#Liberals can be seen differently by some, but let's remember the principle of INNOCENT UNTIL PROVEN GUILTY. It's important to examine both parties if there are concerns about individuals' conduct. Let's strive to have respectful discussions and avoid harsh language. #KavanaughIsInnocent #WalkAwayFromDemocrats. Let's focus on honesty and avoid extreme accusations. URL"


In [75]:
test_tweet_off_new['tweet'][371] ="And sometimes, people can give off the impression that they have money, but in reality, they may not"

<ipython-input-75-a169df788483>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][371] ="And sometimes, people can give off the impression that they have money, but in reality, they may not"


In [76]:
test_tweet_off_new['tweet'][567] ="Ford's attorney has mentioned that she's facing death threats, leading her to relocate her family for safety. However, it's essential to seek evidence before making assumptions about who might be behind these threats. Let's approach this situation with a fair and unbiased perspective"

<ipython-input-76-89db649cffe8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][567] ="Ford's attorney has mentioned that she's facing death threats, leading her to relocate her family for safety. However, it's essential to seek evidence before making assumptions about who might be behind these threats. Let's approach this situation with a fair and unbiased perspective"


In [77]:
test_tweet_off_new['tweet'][428]="Anyone who finds M*neta's behavior humorous may have a different sense of humor."

<ipython-input-77-a3ad3be853f8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][428]="Anyone who finds M*neta's behavior humorous may have a different sense of humor."


In [78]:
test_tweet_off_new['tweet'][740] = "@USER - Opinions about this POTUS vary widely."

<ipython-input-78-2a5c3594618e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][740] = "@USER - Opinions about this POTUS vary widely."


In [79]:
test_tweet_off_new['tweet'][439] = "#CnnIsFakeNews @USER @USER, some people have concerns about the network's ratings and credibility. #cnnisajoke #FakeNews #MAGA URL"

<ipython-input-79-56e972d7dcc8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][439] = "#CnnIsFakeNews @USER @USER, some people have concerns about the network's ratings and credibility. #cnnisajoke #FakeNews #MAGA URL"


In [80]:
test_tweet_off_new['tweet'][408] = "#Hillary and at least 16 other individuals have strong opinions as #AngryDemocrats. Let's discuss #WorldWarQ, #WWG1WGA, #MAGA, #RedTsunami, #AmericaFirst, #WalkAway, and #Truth. URL"

<ipython-input-80-bb9c9e691cfb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][408] = "#Hillary and at least 16 other individuals have strong opinions as #AngryDemocrats. Let's discuss #WorldWarQ, #WWG1WGA, #MAGA, #RedTsunami, #AmericaFirst, #WalkAway, and #Truth. URL"


In [81]:
test_tweet_off_new['tweet'][420] ="@USER @USER As I've mentioned before, some people believe that media and financial influences, like Soros and Waters, are contributing to increased tensions in society."

<ipython-input-81-7fe4dab13301>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][420] ="@USER @USER As I've mentioned before, some people believe that media and financial influences, like Soros and Waters, are contributing to increased tensions in society."


In [82]:
test_tweet_off_new['tweet'][638] ="#Liberals are often known for their strategic tactics to achieve their goals. 👇👇 URL"

<ipython-input-82-8e47485bb2b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][638] ="#Liberals are often known for their strategic tactics to achieve their goals. 👇👇 URL"


In [83]:
test_tweet_off_new['tweet'][663]="#HonourSacrificeBoycottMatch I have concerns about the impact on human rights. I hope all parties involved find some satisfaction. Let's engage in cricket with them and express gratitude for their efforts to support our soldiers. We need to address the situation thoughtfully; even smaller nations like Pakistan should not be underestimated."

<ipython-input-83-cd6a6499042b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][663]="#HonourSacrificeBoycottMatch I have concerns about the impact on human rights. I hope all parties involved find some satisfaction. Let's engage in cricket with them and express gratitude for their efforts to support our soldiers. We need to address the situation thoughtfully; even smaller nations like Pakistan should not be underestimated."


In [84]:
test_tweet_off_new['tweet'][377] = "#Dems have shown a concerning pattern of believing accusations against conservatives while dismissing those against liberals. There is a need for fair and impartial treatment in such cases. URL"

<ipython-input-84-111112012bd0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][377] = "#Dems have shown a concerning pattern of believing accusations against conservatives while dismissing those against liberals. There is a need for fair and impartial treatment in such cases. URL"


In [85]:
test_tweet_off_new['tweet'][775]="#FemiNazis, who are often associated with Liberals, seem to emphasize #MeToo only when accusing non-Liberal men. The situation may seem perplexing. #Bizarro #KAG2018 ❤️🇺🇸 URL"

<ipython-input-85-0829e76ec2de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][775]="#FemiNazis, who are often associated with Liberals, seem to emphasize #MeToo only when accusing non-Liberal men. The situation may seem perplexing. #Bizarro #KAG2018 ❤️🇺🇸 URL"


In [86]:
test_tweet_off_new['tweet'][502] ="@USER One possible explanation is that he may be tone-deaf to the situation. It's challenging to remain understanding when he and others continue to behave this way."

<ipython-input-86-005d457457eb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][502] ="@USER One possible explanation is that he may be tone-deaf to the situation. It's challenging to remain understanding when he and others continue to behave this way."


In [87]:
test_tweet_off_new['tweet'][468] ="#Nigeria #Naija #9ja 'Atiku criticizes Buhari, calling him one of the least competent leaders in Nigeria's history' - Read more on this via URL provided by championnewsng."

<ipython-input-87-a87d5473b1d2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][468] ="#Nigeria #Naija #9ja 'Atiku criticizes Buhari, calling him one of the least competent leaders in Nigeria's history' - Read more on this via URL provided by championnewsng."


In [88]:
test_tweet_off_new['tweet'][177] ="#BlueWave #DumpTrump ...show love and seek to cooperate or find common ground with conservatives/Trump supporters while advocating for the rights of marginalized groups and supporting women's rights to make choices about their own bodies..."

<ipython-input-88-493bd1a9de39>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][177] ="#BlueWave #DumpTrump ...show love and seek to cooperate or find common ground with conservatives/Trump supporters while advocating for the rights of marginalized groups and supporting women's rights to make choices about their own bodies..."


In [89]:
test_tweet_off_new['tweet'][180] ="#NightMayor's DT cycle track can be repurposed for various community activities, like setting up tennis courts for Antifa inside Crescent Heights, until we address concerns related to Nenshi's vape shop."

<ipython-input-89-e9ea4713ab04>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][180] ="#NightMayor's DT cycle track can be repurposed for various community activities, like setting up tennis courts for Antifa inside Crescent Heights, until we address concerns related to Nenshi's vape shop."


In [90]:
test_tweet_off_new['tweet'][422] ="#Texas Ted Cruz's campaign has been sending solicitations that some recipients mistake for official county summons. This has caused confusion, particularly among elderly residents. Some people find this approach questionable. URL"

<ipython-input-90-dc6c4ece8305>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][422] ="#Texas Ted Cruz's campaign has been sending solicitations that some recipients mistake for official county summons. This has caused confusion, particularly among elderly residents. Some people find this approach questionable. URL"


In [91]:
test_tweet_off_new['tweet'][262] = "$uicideboy$ ON HALLOWEEN! Get ready for a seriously spooky time!"

<ipython-input-91-9bf6be0c8b29>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][262] = "$uicideboy$ ON HALLOWEEN! Get ready for a seriously spooky time!"


In [92]:
test_tweet_off_new['tweet'][335] ="#Barbara Boxer: When discussing serious issues like rape, it's crucial for everyone, including liberals, to provide solid evidence. Personal accounts alone might not be sufficient. On a related note, without evidence, making accusations could be misleading. Let's focus on respectful discussions. URL"

<ipython-input-92-15f640ba422e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][335] ="#Barbara Boxer: When discussing serious issues like rape, it's crucial for everyone, including liberals, to provide solid evidence. Personal accounts alone might not be sufficient. On a related note, without evidence, making accusations could be misleading. Let's focus on respectful discussions. URL"


In [93]:
test_tweet_off_new['tweet'][774] ="@USER #Rosie's actions or statements sometimes make me feel uncomfortable. URL"

<ipython-input-93-cfd7cc9c9270>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][774] ="@USER #Rosie's actions or statements sometimes make me feel uncomfortable. URL"


In [94]:
test_tweet_off_new['tweet'][555] ="Anyway, this game isn't really my cup of tea :)"

<ipython-input-94-7cef9a48808e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][555] ="Anyway, this game isn't really my cup of tea :)"


In [95]:
test_tweet_off_new['tweet'][657] ="#Bakersfield is an example of why some people advocate for gun control. My thoughts are with the victims, except for the gunman himself. I hope justice is served"

<ipython-input-95-8c37b4687549>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][657] ="#Bakersfield is an example of why some people advocate for gun control. My thoughts are with the victims, except for the gunman himself. I hope justice is served"


In [96]:
test_tweet_off_new['tweet'][201] ="@USER These seem to be coordinated attacks from individuals with extreme views. It's unfortunate that such incidents may escalate further before the election. Some people criticize the President for not being a traditional politician, even though they fear potential consequences for the economy and their 401 plans. Let's hope for more reasoned and respectful discussions."

<ipython-input-96-8a740da314d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][201] ="@USER These seem to be coordinated attacks from individuals with extreme views. It's unfortunate that such incidents may escalate further before the election. Some people criticize the President for not being a traditional politician, even though they fear potential consequences for the economy and their 401 plans. Let's hope for more reasoned and respectful discussions."


In [97]:
test_tweet_off_new['tweet'][222] ="#PoliticsInFocus Hey Donald! Let's remember we all have different perspectives, no need to talk down to me! URL'"

<ipython-input-97-e2e8d45f3ada>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][222] ="#PoliticsInFocus Hey Donald! Let's remember we all have different perspectives, no need to talk down to me! URL'"


In [98]:
test_tweet_off_new['tweet'][511] ="#Kavanaugh has been influenced by Anthony Kennedy, known for delivering opinions that upset liberals. Some view him as an attempt by Trump to convince conservatives that the #SCOTUS is leaning right. If @USER were strategic, they would work to support BK's confirmation. However, it seems unlikely. #TuesdayThoughts URL"

<ipython-input-98-0688dc202cfa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][511] ="#Kavanaugh has been influenced by Anthony Kennedy, known for delivering opinions that upset liberals. Some view him as an attempt by Trump to convince conservatives that the #SCOTUS is leaning right. If @USER were strategic, they would work to support BK's confirmation. However, it seems unlikely. #TuesdayThoughts URL"


In [99]:
test_tweet_off_new['tweet'][549] ="#NEWS Jeff Sessions: In his perspective, following the guidance of the ACLU, Antifa, and Black Lives Matter may lead to more negative outcomes. URL #CNBC"

<ipython-input-99-886c83ab61e9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][549] ="#NEWS Jeff Sessions: In his perspective, following the guidance of the ACLU, Antifa, and Black Lives Matter may lead to more negative outcomes. URL #CNBC"


In [100]:
test_tweet_off_new['tweet'][509] = "...how about death metal? I'm eager to discover more death metal music... Can you please suggest some good places to find it? URL"

<ipython-input-100-5b56cab57706>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][509] = "...how about death metal? I'm eager to discover more death metal music... Can you please suggest some good places to find it? URL"


In [101]:
test_tweet_off_new['tweet'][587] ="@USER #ThoseThatAreTheLoudest might be concerned, but many believe that justice will prevail. Let's remember to focus on the facts. #Bono and the #ClintonFoundation have raised some questions, and people have expressed skepticism about their actions. #PedoWood #Haiti URL"

<ipython-input-101-a6e4fcd8ff4f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][587] ="@USER #ThoseThatAreTheLoudest might be concerned, but many believe that justice will prevail. Let's remember to focus on the facts. #Bono and the #ClintonFoundation have raised some questions, and people have expressed skepticism about their actions. #PedoWood #Haiti URL"


In [102]:
test_tweet_off_new['tweet'][771] = "#Conservatives might not be as informed as some may hope, that's the main idea behind my tweet."

<ipython-input-102-4a087252eea2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][771] = "#Conservatives might not be as informed as some may hope, that's the main idea behind my tweet."


In [103]:
test_tweet_off_new['tweet'][437] = "#MSNBC #Hardball @USER Bingo, you've hit the mark. It seems evident that he holds a grudge against President Obama, likely stemming from the White House correspondents dinner. This could be a significant factor in his negative feelings towards the former President."

<ipython-input-103-9de7005ad436>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][437] = "#MSNBC #Hardball @USER Bingo, you've hit the mark. It seems evident that he holds a grudge against President Obama, likely stemming from the White House correspondents dinner. This could be a significant factor in his negative feelings towards the former President."


In [104]:
test_tweet_off_new['tweet'][800] ="#Kavanaugh If #BlaseyFord's credibility is questioned, there's a possibility of another accuser coming forward. Some people criticize liberals for their consistent approach in such situations. #LiberalismIsAMentalDisorder"

<ipython-input-104-a1ec25276faa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][800] ="#Kavanaugh If #BlaseyFord's credibility is questioned, there's a possibility of another accuser coming forward. Some people criticize liberals for their consistent approach in such situations. #LiberalismIsAMentalDisorder"


In [105]:
test_tweet_off_new['tweet'][108] = "Amazon is now offering to ship live Christmas trees to your doorstep. But the big question is, will they stay and help with putting on the lights? URL"

<ipython-input-105-fab053333c38>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][108] = "Amazon is now offering to ship live Christmas trees to your doorstep. But the big question is, will they stay and help with putting on the lights? URL"


In [106]:
test_tweet_off_new['tweet'][207] ="#JoinTheFight Join the discussion about concerns regarding certain political ideologies. Some people feel that Democrats and Liberals are promoting ideas that they believe could be harmful to America. Let's engage in constructive conversations. #JoinTheFightAgainstCommunism URL"

<ipython-input-106-b911e7b25eb3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][207] ="#JoinTheFight Join the discussion about concerns regarding certain political ideologies. Some people feel that Democrats and Liberals are promoting ideas that they believe could be harmful to America. Let's engage in constructive conversations. #JoinTheFightAgainstCommunism URL"


In [107]:
test_tweet_off_new['tweet'][609] ="#DeSantis=DeRacist, while some may view him as a questionable choice, he chose to distance himself from the President's comments about deaths in Puerto Rico from Hurricane Maria. @USER @USER #whataputz"

<ipython-input-107-9f32902790f1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][609] ="#DeSantis=DeRacist, while some may view him as a questionable choice, he chose to distance himself from the President's comments about deaths in Puerto Rico from Hurricane Maria. @USER @USER #whataputz"


In [108]:
test_tweet_off_new['tweet'][156] ="#TheArchers I want to excel in Karate and improve my skills for self-defense and personal growth."

<ipython-input-108-a45fbec956cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][156] ="#TheArchers I want to excel in Karate and improve my skills for self-defense and personal growth."


In [210]:
test_tweet_off_new_01 = test_tweet_off_new.loc[index_new]

In [211]:
test_tweet_off_new_01.shape

(50, 2)

In [212]:
test_tweet_not = test_tweet_a_new[test_tweet_a_new['class_a']=='NOT']

In [213]:
random_100_rows = test_tweet_not.sample(n=100)

# Build the new DataFrame using the randomly selected rows
test_tweet_not_new = pd.DataFrame(random_100_rows)

In [371]:
test_tweet_not_new

,tweet,class_a
114,"*Available for Adoption* Lincoln County, NC Ro...",NOT
246,@USER @USER @USER You have yet to answer what ...,NOT
696,@USER #Google as a fellow Goggler I must say e...,NOT
193,"#1A #2A Ok,@USER , @USER This is Eric Holder...",NOT
379,#CREW is an eminence front. It's a put-on. #MS...,NOT
...,...,...
805,#ChristineBlaseyFord / Democrats condemn conse...,NOT
74,@USER Well she is Chuck Schumer's cousin or ni...,NOT
794,#Wisdom #Wellness I will teach you the right w...,NOT
451,#MyUniqueHeritage I Get My Irish From My Dad S...,NOT


In [215]:
merged_df = pd.concat([test_tweet_not_new, test_tweet_off_new_01], ignore_index=True)

# Shuffle the index
test_shuffled_df = merged_df.sample(frac=1).reset_index(drop=True)

In [216]:
test_shuffled_df

,tweet,class_a
0,#Christian If Twitter and Facebook can silence...,NOT
1,#Antifa demo in memory of #KillahP. #Greece URL,NOT
2,#MyUniqueHeritage I have a separate family tr...,NOT
3,"@USER @USER As I've mentioned before, some peo...",OFF
4,#Antifa invaded a memorial in recognition of K...,NOT
...,...,...
145,@USER He does. And I'm glad he always goes out...,NOT
146,69. he managed it... URL,NOT
147,#TipOfTheDay If you are giving #coffee as a gi...,NOT
148,#Trademark Attorney Daniela Paull will be in P...,NOT


In [217]:
clean_tweets(test_shuffled_df)

<ipython-input-10-58d0d99cd0a4>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
<ipython-input-10-58d0d99cd0a4>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')


In [ ]:

test_shuffled_df.update(test_shuffled_df[['tweet']].applymap('\'{}\''.format))

test_shuffled_df.head()

In [219]:

test_shuffled_df["class_a_code"] = encoder.fit_transform(test_shuffled_df["class_a"])

In [220]:
X_test_shuffled = test_shuffled_df['tweet']
y_test_shuffled = test_shuffled_df['class_a_code']

In [221]:
X_test_shuffled

0      'christian if twitter and facebook can silence...
1             'antifa demo in memory of killahp greece '
2      'myuniqueheritage  i have a separate family tr...
3      '  as ive mentioned before some people believe...
4      'antifa invaded a memorial in recognition of k...
                             ...                        
145    ' he does and im glad he always goes out on so...
146                                    ' he managed it '
147    'tipoftheday if you are giving coffee as a gif...
148    'trademark attorney daniela paull will be in p...
149    'serenawilliams making that  start to this st ...
Name: tweet, Length: 150, dtype: object

In [222]:
y_test_shuffled

0      0
1      0
2      0
3      1
4      0
      ..
145    0
146    0
147    0
148    0
149    0
Name: class_a_code, Length: 150, dtype: int64

In [223]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_shuffled = encoder.transform(y_test_shuffled)

In [224]:
test_encodings_shuffled = tokenizer(list(X_test_shuffled), truncation=True, padding=True, max_length=256)

In [225]:
test_dataset_shuffled = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_shuffled),
    test_labels_shuffled
)).batch(16)

In [226]:

# Get the model predictions
predictions = tf.argmax(model.predict(test_dataset_shuffled)[0], axis=-1)


10/10 [==============================] - 1s 103ms/step


In [227]:
# Decode the label numbers back into original labels
y_test_inverse_shuffled = encoder.inverse_transform(y_test_shuffled)
predictions_inverse_shuffled = encoder.inverse_transform(predictions.numpy())

# Generate the classification report
report_gpt3 = classification_report(y_test_inverse_shuffled, predictions_inverse_shuffled,
                               target_names=encoder.classes_.astype(str))
print(report_gpt3)

              precision    recall  f1-score   support

           0       0.63      0.85      0.73       100
           1       0.06      0.02      0.03        50

    accuracy                           0.57       150
   macro avg       0.35      0.43      0.38       150
weighted avg       0.44      0.57      0.49       150



# ChatGPT 3 with rephrasing


In [ ]:
print(random_indexes[50:])

[308, 216, 732, 376, 382, 339, 47, 99, 797, 282, 228, 362, 701, 480, 115, 136, 77, 51, 681, 749, 389, 260, 258, 100, 233, 554, 810, 483, 614, 778, 107, 613, 175, 307, 634, 313, 182, 154, 855, 352, 853, 323, 40, 38, 531, 135, 281, 737, 823, 413]


In [149]:
index_new2 = [308, 216, 732, 376, 382, 339, 47, 99, 797, 282, 228, 362, 701, 480, 115, 136, 77, 51, 681, 749, 389, 260, 258, 100, 233, 554, 810, 483, 614, 778, 107, 613, 175, 307, 634, 313, 182, 154, 855, 352, 853, 323, 40, 38, 531, 135, 281, 737, 823, 413]

In [150]:
test_tweet_off_new['tweet'][413]

'#Antifa is recruiting for a designated #terrorist organization URL'

In [151]:
test_tweet_off_new['tweet'][308] = "Alright, let's consider the situation. It's true that some players have scored 15+ goals in Eredivisie, but some might argue that it's not the most challenging league. On the other hand, Bundesliga teams have proven to be competitive in European competitions. As for Mexican talent, it's been some time since someone like Pulisic emerged. Nevertheless, there's always hope for the future, and who knows, Mexico might produce exceptional talents in due time."

<ipython-input-151-0915c0305988>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][308] = "Alright, let's consider the situation. It's true that some players have scored 15+ goals in Eredivisie, but some might argue that it's not the most challenging league. On the other hand, Bundesliga teams have proven to be competitive in European competitions. As for Mexican talent, it's been some time since someone like Pulisic emerged. Nevertheless, there's always hope for the future, and who knows, Mexico might produce exceptional talents in due time."


In [152]:
test_tweet_off_new['tweet'][216] ="It seems like both males and females are dealing with similar situations, and that's something to consider. The URL might be a shared experience for both genders."

<ipython-input-152-83836923c3a5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][216] ="It seems like both males and females are dealing with similar situations, and that's something to consider. The URL might be a shared experience for both genders."


In [153]:
test_tweet_off_new['tweet'][732] = "Addressing the topic of #LiberalHypocrisy and #TacoBell, some individuals question why others are against illegal immigration. They mention an incident involving a Taco Bell employee responding with 'No Habla Ingles!' when asked."

<ipython-input-153-f98cb3a6eb0b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][732] = "Addressing the topic of #LiberalHypocrisy and #TacoBell, some individuals question why others are against illegal immigration. They mention an incident involving a Taco Bell employee responding with 'No Habla Ingles!' when asked."


In [154]:
test_tweet_off_new['tweet'][376]  ="It's been an incredible seven years! #MyTwitterAnniversary URL"

<ipython-input-154-5bd8c908bd1d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][376]  ="It's been an incredible seven years! #MyTwitterAnniversary URL"


In [155]:
test_tweet_off_new['tweet'][382] ="The situation surrounding #ChristineBlaseyFord has drawn attention, and some suggest that she is facing harassment from certain groups. They point to the involvement of Antifa, seen as the radical arm of the Democratic Party, allegedly backed by George Soros. The hashtag #VoteRedToSaveAmerica is being used in this context."

<ipython-input-155-faf8206244da>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][382] ="The situation surrounding #ChristineBlaseyFord has drawn attention, and some suggest that she is facing harassment from certain groups. They point to the involvement of Antifa, seen as the radical arm of the Democratic Party, allegedly backed by George Soros. The hashtag #VoteRedToSaveAmerica is being used in this context."


In [156]:
test_tweet_off_new['tweet'][339] ="On #ConstitutionDay, there's an interesting contrast between some conservative members of the alt-right who criticize efforts for #GunControlNow, accusing it of damaging the Constitution, while expressing concerns about non-White foreigners seeking to live in this country. The situation sparks debate on constitutional interpretations and immigration policies."

<ipython-input-156-ca602e3bb761>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][339] ="On #ConstitutionDay, there's an interesting contrast between some conservative members of the alt-right who criticize efforts for #GunControlNow, accusing it of damaging the Constitution, while expressing concerns about non-White foreigners seeking to live in this country. The situation sparks debate on constitutional interpretations and immigration policies."


In [157]:
test_tweet_off_new['tweet'][47] ="#Conservatives, it's disheartening to witness such disrespectful interactions like the one with @USER. URL"

<ipython-input-157-4145a0397fcd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][47] ="#Conservatives, it's disheartening to witness such disrespectful interactions like the one with @USER. URL"


In [158]:
test_tweet_off_new['tweet'][99]  ="Some people find #SerenaWilliams to be overly confident, making it challenging to enjoy both watching her play and listening to her speak."

<ipython-input-158-c8c8e47abdcd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][99]  ="Some people find #SerenaWilliams to be overly confident, making it challenging to enjoy both watching her play and listening to her speak."


In [159]:
test_tweet_off_new['tweet'][797] ="If you want to encourage people to leave your home, perhaps you could try using humor or lighthearted jokes, like pretending to be a vampire, without causing harm or discomfort. It's essential to find a friendly and non-threatening way to address the situation."

<ipython-input-159-7caf5b2c333e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][797] ="If you want to encourage people to leave your home, perhaps you could try using humor or lighthearted jokes, like pretending to be a vampire, without causing harm or discomfort. It's essential to find a friendly and non-threatening way to address the situation."


In [160]:
test_tweet_off_new['tweet'][282] = "The ongoing #CrowderConfronts series continues to demonstrate that when conservatives respond with truth and facts, the so-called SJW bullies appear less intimidating and substantial. It encourages others not to be afraid and emphasizes the importance of confronting issues with honesty and evidence. URL"

<ipython-input-160-fabc9cee5965>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][282] = "The ongoing #CrowderConfronts series continues to demonstrate that when conservatives respond with truth and facts, the so-called SJW bullies appear less intimidating and substantial. It encourages others not to be afraid and emphasizes the importance of confronting issues with honesty and evidence. URL"


In [161]:
test_tweet_off_new['tweet'][228] ="'#CarbonTax proves effective and promises to significantly increase money in the pockets of Canadians. .@USER .@USER .@USER .@USER Back in 2015, part of Trudeau's successful campaign was centered around a national carbon tax, which played a role in the Conservative party's defeat. They face a similar challenge in 2019, and some are determined to ensure their loss once again. URL'"

<ipython-input-161-00924abefeee>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][228] ="'#CarbonTax proves effective and promises to significantly increase money in the pockets of Canadians. .@USER .@USER .@USER .@USER Back in 2015, part of Trudeau's successful campaign was centered around a national carbon tax, which played a role in the Conservative party's defeat. They face a similar challenge in 2019, and some are determined to ensure their loss once again. URL'"


In [162]:
test_tweet_off_new['tweet'][362] ="'#Repost theleadersmindset • • • 📍Hello Leaders!🔥 ✔️Type 'READY' if you are prepared to embrace these steps. - 'Set goals, keep them to yourself, crush them with determination, celebrate your successes… URL'"

<ipython-input-162-f9778d3b69b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][362] ="'#Repost theleadersmindset • • • 📍Hello Leaders!🔥 ✔️Type 'READY' if you are prepared to embrace these steps. - 'Set goals, keep them to yourself, crush them with determination, celebrate your successes… URL'"


In [163]:
test_tweet_off_new['tweet'][701] ="'@USER nigga are you stupid your trash dont play with him play with your bitch 😂'"

<ipython-input-163-025c3160f613>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][701] ="'@USER nigga are you stupid your trash dont play with him play with your bitch 😂'"


In [164]:
test_tweet_off_new['tweet'][480] ="#FailedDemFeinstein's continued presence in politics is questionable, and some believe she should have stepped down years ago. Her recent actions, like blaming the GOP in the Kavanaugh accuser situation while admitting uncertainty about the truthfulness, add to her perceived disgrace. URL"

<ipython-input-164-b324e4413ff0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][480] ="#FailedDemFeinstein's continued presence in politics is questionable, and some believe she should have stepped down years ago. Her recent actions, like blaming the GOP in the Kavanaugh accuser situation while admitting uncertainty about the truthfulness, add to her perceived disgrace. URL"


In [165]:
test_tweet_off_new['tweet'][115] ="#Conservatives Govt ' @USER caused immense hardship': whistleblowers report being threatened by a regulator; Weren't they supposed to safeguard whistleblowers? URL"

<ipython-input-165-146d275d4f9c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][115] ="#Conservatives Govt ' @USER caused immense hardship': whistleblowers report being threatened by a regulator; Weren't they supposed to safeguard whistleblowers? URL"


In [166]:
test_tweet_off_new['tweet'][136] ="@USER #WalkAwayFromAllDemocrats Michelle had her 8 years, and many believe we don't need more of her advice. We now have a new Sheriff in Town who's #MAGA. It's crucial for everyone, regardless of color or gender (AKA all Americans), to get out and VOTE! URL"

<ipython-input-166-81f13225217d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][136] ="@USER #WalkAwayFromAllDemocrats Michelle had her 8 years, and many believe we don't need more of her advice. We now have a new Sheriff in Town who's #MAGA. It's crucial for everyone, regardless of color or gender (AKA all Americans), to get out and VOTE! URL"


In [167]:
test_tweet_off_new['tweet'][77] ="#ANTIFA is seen as allied with The Democratic Socialists of America (DSA), whom #JamesOKeefe has revealed as having infiltrated various government positions with the aim of undermining #America. #Trump #MAGA🇺🇸 #DeepState URL"

<ipython-input-167-d3f41301cad6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][77] ="#ANTIFA is seen as allied with The Democratic Socialists of America (DSA), whom #JamesOKeefe has revealed as having infiltrated various government positions with the aim of undermining #America. #Trump #MAGA🇺🇸 #DeepState URL"


In [168]:
test_tweet_off_new['tweet'][51] ="'#Kids Beating #Paraplegic #Trump #Boy!  #Left Teach Their #Children Hate Violence!  #schools #teachers #Democrats #Liberals #WalkAway #Moms #Mother #NEA #teachthemyoung #Christian #Catholic #Jews #Evangelicals #family #BlackWomen #BlackTwitter #Blacks #News #Media #Disabled #MAGA URL'"

<ipython-input-168-aa9f57587ed5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][51] ="'#Kids Beating #Paraplegic #Trump #Boy!  #Left Teach Their #Children Hate Violence!  #schools #teachers #Democrats #Liberals #WalkAway #Moms #Mother #NEA #teachthemyoung #Christian #Catholic #Jews #Evangelicals #family #BlackWomen #BlackTwitter #Blacks #News #Media #Disabled #MAGA URL'"


In [169]:
test_tweet_off_new['tweet'][681] ="Can you believe it? A hundred rounds! Goon, you must be kidding me! 🤣"

<ipython-input-169-de5768d9c24f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][681] ="Can you believe it? A hundred rounds! Goon, you must be kidding me! 🤣"


In [170]:
test_tweet_off_new['tweet'][749] ="#Antifa plans to counter the fascist NVU in #Amsterdam on September 22. URL"

<ipython-input-170-8dba53670cd6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][749] ="#Antifa plans to counter the fascist NVU in #Amsterdam on September 22. URL"


In [171]:
test_tweet_off_new['tweet'][389] ="50 Cent addresses Joe Budden's actions on Instagram, calling them out as 'bullshit.' URL URL"

<ipython-input-171-b24dea14c11b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][389] ="50 Cent addresses Joe Budden's actions on Instagram, calling them out as 'bullshit.' URL URL"


In [172]:
test_tweet_off_new['tweet'][260] ="#Liberals are resorting to extreme measures, urging #PhillipRuddock to communicate with #Turnbull and persuade him to assist with #WentworthVotes on 18 Sept 2018. @USER #Auspol #LNP #NSWpol @USER @USER @USER #LNPMemes URL"

<ipython-input-172-25d11035b2d3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][260] ="#Liberals are resorting to extreme measures, urging #PhillipRuddock to communicate with #Turnbull and persuade him to assist with #WentworthVotes on 18 Sept 2018. @USER #Auspol #LNP #NSWpol @USER @USER @USER #LNPMemes URL"


In [173]:
test_tweet_off_new['tweet'][258] ="'#CincinnatiEnquirer finally addresses #GunControl following a tragic #MassShooting right on their doorstep. It took this event for their editors to wake up, but better late than never. URL'"

<ipython-input-173-c63e0cd04932>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][258] ="'#CincinnatiEnquirer finally addresses #GunControl following a tragic #MassShooting right on their doorstep. It took this event for their editors to wake up, but better late than never. URL'"


In [174]:
test_tweet_off_new['tweet'][100] ="Regarding #ColinKaepernick, some people believe he has been unproductive and merely voicing complaints. They think it's time for both black individuals and passionate white liberals, including @USER, to become more aware or stop making noise about the cause. They see Colin as not fitting the role of a Civil Rights hero or martyr by any means."

<ipython-input-174-dbfbe86d7102>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][100] ="Regarding #ColinKaepernick, some people believe he has been unproductive and merely voicing complaints. They think it's time for both black individuals and passionate white liberals, including @USER, to become more aware or stop making noise about the cause. They see Colin as not fitting the role of a Civil Rights hero or martyr by any means."


In [175]:
test_tweet_off_new['tweet'][233] ="During #TheView, Stormy Daniels found herself in a difficult situation with a dollar bill in her face, highlighting the challenges faced by adult film stars. The post also seems to allude to her involvement with the Democratic Party."

<ipython-input-175-d7b2bd5360d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][233] ="During #TheView, Stormy Daniels found herself in a difficult situation with a dollar bill in her face, highlighting the challenges faced by adult film stars. The post also seems to allude to her involvement with the Democratic Party."


In [176]:
test_tweet_off_new['tweet'][554] = "#DavidMiliband relocated to #NewYork to align with #US #Liberals #Democrats, seeking a departure from the hostility towards Jews by #Corbynists."

<ipython-input-176-c5a1bd386006>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][554] = "#DavidMiliband relocated to #NewYork to align with #US #Liberals #Democrats, seeking a departure from the hostility towards Jews by #Corbynists."


In [177]:
test_tweet_off_new['tweet'][810] ="'#BCOG @USER portrays a cool, confident, and assertive character who is highly skilled and successful! @USER @USER @USER URL'"

<ipython-input-177-e8bb05806958>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][810] ="'#BCOG @USER portrays a cool, confident, and assertive character who is highly skilled and successful! @USER @USER @USER URL'"


In [178]:
test_tweet_off_new['tweet'][483] ="'@USER @USER @USER Your meme suggests that both Fascism and Antifa were opposed to liberals.'"

<ipython-input-178-1bfbb20d2c8a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][483] ="'@USER @USER @USER Your meme suggests that both Fascism and Antifa were opposed to liberals.'"


In [179]:
test_tweet_off_new['tweet'][614] ="@USER @USER I plan to use that argument in future discussions on gun control, free speech, or taxes. It's essential to promote responsibility, and that's what I advocate for. There's nothing right about justifying murder by denying the life of another or claiming it as a right."

<ipython-input-179-5131a9741633>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][614] ="@USER @USER I plan to use that argument in future discussions on gun control, free speech, or taxes. It's essential to promote responsibility, and that's what I advocate for. There's nothing right about justifying murder by denying the life of another or claiming it as a right."


In [180]:
test_tweet_off_new['tweet'][778] ="It appears that a #BREXIT deal has been reached and will be presented at a special summit in November. The question raised is whether @USER has compromised the #UK's position with the EU. People are concerned, hoping that there hasn't been a sellout, as it could have significant consequences for @USER. URL"

<ipython-input-180-927056a2da8b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][778] ="It appears that a #BREXIT deal has been reached and will be presented at a special summit in November. The question raised is whether @USER has compromised the #UK's position with the EU. People are concerned, hoping that there hasn't been a sellout, as it could have significant consequences for @USER. URL"


In [181]:
test_tweet_off_new['tweet'][107] ="Jeff Sessions suggests that listening to groups like the ACLU, Antifa, and Black Lives Matter could result in an increase in Chicago shootings. URL"

<ipython-input-181-43a5579b2dae>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][107] ="Jeff Sessions suggests that listening to groups like the ACLU, Antifa, and Black Lives Matter could result in an increase in Chicago shootings. URL"


In [182]:
test_tweet_off_new['tweet'][613] ="Lost my acca on the first fight, 0-1. So frustrating."

<ipython-input-182-0bc83ebdcb8f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][613] ="Lost my acca on the first fight, 0-1. So frustrating."


In [183]:
test_tweet_off_new['tweet'][307] ="#1 son, hitting it out of the park! URL"

<ipython-input-183-742226f77f65>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][307] ="#1 son, hitting it out of the park! URL"


In [184]:
test_tweet_off_new['tweet'][313]  = "@USER @USER @USER Many believe she is the most disingenuous person in the Senate. If she were my Senator, I would feel ashamed due to her despicable behavior in the SCOTUS hearings."

<ipython-input-184-c2f32f0e240f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][313]  = "@USER @USER @USER Many believe she is the most disingenuous person in the Senate. If she were my Senator, I would feel ashamed due to her despicable behavior in the SCOTUS hearings."


In [185]:
test_tweet_off_new['tweet'][182] = "#ArunJaitleyStepDown is trending, with many expressing strong criticism towards him. They consider him the most shameless Finance Minister in the history of India, and his audacity to lie in public is viewed as a disgrace to his position."

<ipython-input-185-7980f575acdd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][182] = "#ArunJaitleyStepDown is trending, with many expressing strong criticism towards him. They consider him the most shameless Finance Minister in the history of India, and his audacity to lie in public is viewed as a disgrace to his position."


In [186]:
test_tweet_off_new['tweet'][154] ="#GOP still appearing hesitant, and some view them as lacking courage. It seems they haven't realized that Liberals will oppose them no matter what they say. Some believe Republicans are either struggling to make strategic decisions or have shifted too far left. #ConfirmKavanaughNow"

<ipython-input-186-54fd127589ad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][154] ="#GOP still appearing hesitant, and some view them as lacking courage. It seems they haven't realized that Liberals will oppose them no matter what they say. Some believe Republicans are either struggling to make strategic decisions or have shifted too far left. #ConfirmKavanaughNow"


In [187]:
test_tweet_off_new['tweet'][855] = "'#DespicableDems are accused of spreading misinformation about rifles. Some claim that Democrats manipulated the law to advocate for gun control during the Kavanaugh confirmation. URL via @USER'"

<ipython-input-187-42d5f31fceec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][855] = "'#DespicableDems are accused of spreading misinformation about rifles. Some claim that Democrats manipulated the law to advocate for gun control during the Kavanaugh confirmation. URL via @USER'"


In [188]:
test_tweet_off_new['tweet'][352] = "'#OrrinHatch's involvement in another woman's charges against a Supreme Court nominee has sparked strong criticism. Some view him as sexist, clueless, and outdated, drawing comparisons to his past behavior during the Anita Hill case. They believe it's time for him to step aside. Good riddance to this Neanderthal!'"

<ipython-input-188-1d8b46a4f44e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][352] = "'#OrrinHatch's involvement in another woman's charges against a Supreme Court nominee has sparked strong criticism. Some view him as sexist, clueless, and outdated, drawing comparisons to his past behavior during the Anita Hill case. They believe it's time for him to step aside. Good riddance to this Neanderthal!'"


In [189]:
test_tweet_off_new['tweet'][853] ="Both of them have an unpleasant taste. URL"

<ipython-input-189-eead7fa6308c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][853] ="Both of them have an unpleasant taste. URL"


In [190]:
test_tweet_off_new['tweet'][323] = "#AVALANCHE, I apologize! I acknowledge your presence, and I care about you! I regret my previous behavior and acknowledge that we all make mistakes. #maga"

<ipython-input-190-c55620d129c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][323] = "#AVALANCHE, I apologize! I acknowledge your presence, and I care about you! I regret my previous behavior and acknowledge that we all make mistakes. #maga"


In [191]:
test_tweet_off_new['tweet'][38] = "#ArianaAsesina? Seriously?! Please, let's not jump to conclusions and spread hate. She's already going through a tough time, and pointing fingers won't help. It was an overdose, and blaming others won't change the tragic outcome. Let's stop seeking someone to blame, and instead, show empathy and support."



<ipython-input-191-2a2c2cbf7bf5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][38] = "#ArianaAsesina? Seriously?! Please, let's not jump to conclusions and spread hate. She's already going through a tough time, and pointing fingers won't help. It was an overdose, and blaming others won't change the tragic outcome. Let's stop seeking someone to blame, and instead, show empathy and support."


In [192]:
test_tweet_off_new['tweet'][531] = "@USER, it's widely known that a genuine leader doesn't stumble or fall like a sack of potatoes into a van..."

<ipython-input-192-7b7ba3f32bef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][531] = "@USER, it's widely known that a genuine leader doesn't stumble or fall like a sack of potatoes into a van..."


In [193]:
test_tweet_off_new['tweet'][135] = "'#BoycottNFL, I have decided to no longer support you. You have lost my loyalty. @USER @USER URL'"

<ipython-input-193-b4f488f4c581>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][135] = "'#BoycottNFL, I have decided to no longer support you. You have lost my loyalty. @USER @USER URL'"


In [194]:
test_tweet_off_new['tweet'][281]  = "'#Conservatives Govt has accumulated debt despite implementing austerity cuts, and during this time, the wealth of the rich has doubled. This highlights the issue of #inequality. URL via @USER'"

<ipython-input-194-16adc7a9858f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][281]  = "'#Conservatives Govt has accumulated debt despite implementing austerity cuts, and during this time, the wealth of the rich has doubled. This highlights the issue of #inequality. URL via @USER'"


In [195]:
test_tweet_off_new['tweet'][413] = "'#Antifa is actively seeking new members for what some perceive as a designated #terrorist organization. URL'"

<ipython-input-195-e0257bfed19e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new['tweet'][413] = "'#Antifa is actively seeking new members for what some perceive as a designated #terrorist organization. URL'"


In [197]:
test_tweet_off_new_02 = test_tweet_off_new.loc[index_new2]


In [198]:

merged_df_02 = pd.concat([test_tweet_not_new, test_tweet_off_new_02], ignore_index=True)

# Shuffle the index
test_02_df = merged_df_02.sample(frac=1,random_state=42).reset_index(drop=True)

In [199]:
test_02_df

,tweet,class_a
0,#RealSmart Answer: This is step #1 on your #Re...,NOT
1,#StayOnYourPath #HealthQuote If you are wishin...,NOT
2,"Can you believe it? A hundred rounds! Goon, yo...",OFF
3,#Spaldo is my guy. He is my godson,NOT
4,#Preach #BadCompany Don’t you know that you ar...,NOT
...,...,...
145,69. he managed it... URL,NOT
146,"#Conservatives, it's disheartening to witness ...",OFF
147,#MyUniqueHeritage I Get My Irish From My Dad S...,NOT
148,9/9/2018 7:00 PM Temp 65.4 Dew Pt 51.1 Barome...,NOT


In [200]:
clean_tweets(test_02_df)

<ipython-input-10-58d0d99cd0a4>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
<ipython-input-10-58d0d99cd0a4>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')


In [201]:
test_02_df.update(test_02_df[['tweet']].applymap('\'{}\''.format))

test_02_df.head()

,tweet,class_a
0,'realsmart answer this is step on your realsm...,NOT
1,'stayonyourpath healthquote if you are wishing...,NOT
2,'can you believe it a hundred rounds goon you ...,OFF
3,'spaldo is my guy he is my godson',NOT
4,'preach badcompany dont you know that you are ...,NOT


In [202]:

test_02_df["class_a_code"] = encoder.fit_transform(test_02_df["class_a"])

In [203]:
X_test_02 = test_02_df['tweet']
y_test_02 = test_02_df['class_a_code']

In [204]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_02 = encoder.transform(y_test_02)

In [205]:
test_encodings_02 = tokenizer(list(X_test_02), truncation=True, padding=True, max_length=256)

In [206]:
test_dataset_02 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_02),
    test_labels_02
)).batch(16)

In [207]:
# Get the model predictions
predictions_02 = tf.argmax(model.predict(test_dataset_02)[0], axis=-1)

10/10 [==============================] - 1s 95ms/step


In [208]:
# Decode the label numbers back into original labels
y_test_inverse_02 = encoder.inverse_transform(y_test_02)
predictions_inverse_02 = encoder.inverse_transform(predictions_02.numpy())

# Generate the classification report
report_02 = classification_report(y_test_inverse_02, predictions_inverse_02,
                               target_names=encoder.classes_.astype(str))
print(report_02)

              precision    recall  f1-score   support

           0       0.71      0.82      0.76       100
           1       0.47      0.32      0.38        50

    accuracy                           0.65       150
   macro avg       0.59      0.57      0.57       150
weighted avg       0.63      0.65      0.63       150



# ChatGPT 4 with rephrasing

In [230]:
index_new3 = random.sample(index_new2, 25)

In [232]:
print(index_new3)


[135, 216, 308, 389, 823, 115, 323, 77, 260, 281, 352, 47, 233, 38, 614, 100, 376, 681, 797, 749, 362, 483, 313, 382, 853]


In [396]:
test_tweet_off_new_4 = test_tweet_off

In [397]:
test_tweet_off_new_4['tweet'][853]


'Both possess a displeasing flavor. [URL].'

In [398]:
test_tweet_off_new_4['tweet'][135] ="#AvoidNFL, I've chosen to withdraw my support. My loyalty has been lost. @USER @USER [URL]"

<ipython-input-398-409e460aba72>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][135] ="#AvoidNFL, I've chosen to withdraw my support. My loyalty has been lost. @USER @USER [URL]"


In [399]:
test_tweet_off_new_4['tweet'][216] = "Both men and women appear to face similar circumstances, which is worth noting. The URL could reflect an experience common to both genders."

<ipython-input-399-b1ce6567346d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][216] = "Both men and women appear to face similar circumstances, which is worth noting. The URL could reflect an experience common to both genders."


In [400]:
test_tweet_off_new_4['tweet'][308] ="Okay, let's evaluate the scenario. While certain players have netted more than 15 goals in the Eredivisie, there are those who believe it's not the toughest league. Conversely, Bundesliga squads have demonstrated their mettle in European tournaments. Regarding Mexican players, it's been a while since we've seen a standout like Pulisic. However, the future remains promising, and Mexico could very well nurture outstanding players in the coming years."

<ipython-input-400-0069563c5158>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][308] ="Okay, let's evaluate the scenario. While certain players have netted more than 15 goals in the Eredivisie, there are those who believe it's not the toughest league. Conversely, Bundesliga squads have demonstrated their mettle in European tournaments. Regarding Mexican players, it's been a while since we've seen a standout like Pulisic. However, the future remains promising, and Mexico could very well nurture outstanding players in the coming years."


In [401]:
test_tweet_off_new_4['tweet'][389] ="50 Cent calls out Joe Budden's behavior on Instagram, labeling it as 'nonsense.' [URL] [URL]"

<ipython-input-401-1db90cad2311>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][389] ="50 Cent calls out Joe Budden's behavior on Instagram, labeling it as 'nonsense.' [URL] [URL]"


In [402]:
test_tweet_off_new_4['tweet'][823] = "#DemocraticPartyConcerns #Extremists #AntifaIssues #RadicalFeminism Consider distancing yourself. [URL]"

<ipython-input-402-ec6c14a14322>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][823] = "#DemocraticPartyConcerns #Extremists #AntifaIssues #RadicalFeminism Consider distancing yourself. [URL]"


In [403]:
test_tweet_off_new_4['tweet'][115] = "#ConservativeGovernment - Whistleblowers claim they faced threats under @USER's regime, despite regulators supposed to protect them. [URL]"

<ipython-input-403-53c5b7c3a729>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][115] = "#ConservativeGovernment - Whistleblowers claim they faced threats under @USER's regime, despite regulators supposed to protect them. [URL]"


In [404]:
test_tweet_off_new_4['tweet'][323] ="#AVALANCHE, my sincere apologies! I recognize and value you. Regretfully, I made errors in the past, but we all err at times. #maga"

<ipython-input-404-f0b2af06cff9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][323] ="#AVALANCHE, my sincere apologies! I recognize and value you. Regretfully, I made errors in the past, but we all err at times. #maga"


In [405]:
test_tweet_off_new_4['tweet'][77] = "#ANTIFA is perceived to be associated with The Democratic Socialists of America (DSA). #JamesOKeefe has exposed their alleged presence in multiple government roles, purportedly aiming to challenge #America's foundations. #Trump #MAGA🇺🇸 #UnderlyingForces [URL]"

<ipython-input-405-cd76d7daff33>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][77] = "#ANTIFA is perceived to be associated with The Democratic Socialists of America (DSA). #JamesOKeefe has exposed their alleged presence in multiple government roles, purportedly aiming to challenge #America's foundations. #Trump #MAGA🇺🇸 #UnderlyingForces [URL]"


In [406]:
test_tweet_off_new_4['tweet'][260] ="#Liberals are taking significant steps, asking #PhillipRuddock to liaise with #Turnbull in hopes of influencing the #WentworthVotes on 18 Sept 2018. @USER #Auspol #LNP #NSWpol @USER @USER @USER #LNPTrends [URL]"

<ipython-input-406-62aa1866cab2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][260] ="#Liberals are taking significant steps, asking #PhillipRuddock to liaise with #Turnbull in hopes of influencing the #WentworthVotes on 18 Sept 2018. @USER #Auspol #LNP #NSWpol @USER @USER @USER #LNPTrends [URL]"


In [407]:
test_tweet_off_new_4['tweet'][281] ="#Conservatives Government, despite austerity measures, has seen an increase in debt. Meanwhile, the affluent's wealth has grown twofold, spotlighting the #inequality problem. [URL] courtesy of @USER."

<ipython-input-407-2bd399889584>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][281] ="#Conservatives Government, despite austerity measures, has seen an increase in debt. Meanwhile, the affluent's wealth has grown twofold, spotlighting the #inequality problem. [URL] courtesy of @USER."


In [408]:
test_tweet_off_new_4['tweet'][352]  ="#OrrinHatch's role in handling another woman's allegations against a Supreme Court nominee has garnered significant backlash. Some consider his approach reminiscent of his stance during the Anita Hill case, labeling him as out of touch. Many feel it's time for a change in leadership."

<ipython-input-408-c4649bf75655>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][352]  ="#OrrinHatch's role in handling another woman's allegations against a Supreme Court nominee has garnered significant backlash. Some consider his approach reminiscent of his stance during the Anita Hill case, labeling him as out of touch. Many feel it's time for a change in leadership."


In [409]:
test_tweet_off_new_4['tweet'][47]  ="#Conservatives, it's unfortunate to see such discourteous exchanges, especially ones involving @USER. [URL]"

<ipython-input-409-1ada6eb16484>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][47]  ="#Conservatives, it's unfortunate to see such discourteous exchanges, especially ones involving @USER. [URL]"


In [410]:
test_tweet_off_new_4['tweet'][233] ="On #TheView, Stormy Daniels encountered a challenging moment when confronted with a dollar bill, underscoring the difficulties adult film actors often face. The mention also hints at her ties to the Democratic Party."

<ipython-input-410-858ee1c19dad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][233] ="On #TheView, Stormy Daniels encountered a challenging moment when confronted with a dollar bill, underscoring the difficulties adult film actors often face. The mention also hints at her ties to the Democratic Party."


In [411]:
test_tweet_off_new_4['tweet'][38] ="Using the hashtag #ArianaAsesina? Let's avoid rushing to judgments and promoting negativity. She's facing difficult moments, and assigning blame doesn't alter the sad reality. An overdose occurred, and finding fault won't help. Let's prioritize understanding and compassion over blame."

<ipython-input-411-6c15cf7b16b4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][38] ="Using the hashtag #ArianaAsesina? Let's avoid rushing to judgments and promoting negativity. She's facing difficult moments, and assigning blame doesn't alter the sad reality. An overdose occurred, and finding fault won't help. Let's prioritize understanding and compassion over blame."


In [412]:
test_tweet_off_new_4['tweet'][614] ="@USER @USER I intend to incorporate that perspective in upcoming debates about gun control, free speech, and taxes. Advocating for responsibility is crucial. It's incorrect to excuse taking a life by negating its value or asserting it as an entitlement."

<ipython-input-412-f3d199926d3b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][614] ="@USER @USER I intend to incorporate that perspective in upcoming debates about gun control, free speech, and taxes. Advocating for responsibility is crucial. It's incorrect to excuse taking a life by negating its value or asserting it as an entitlement."


In [413]:
test_tweet_off_new_4['tweet'][100] = "Concerning #ColinKaepernick, there's a viewpoint that he has been more vocal than effective. Some argue that both the black community and supportive white liberals, including @USER, should be more informed or reduce the vocal intensity about the issue. They don't see Colin as embodying the image of a Civil Rights champion."

<ipython-input-413-7dcbde48dead>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][100] = "Concerning #ColinKaepernick, there's a viewpoint that he has been more vocal than effective. Some argue that both the black community and supportive white liberals, including @USER, should be more informed or reduce the vocal intensity about the issue. They don't see Colin as embodying the image of a Civil Rights champion."


In [414]:
test_tweet_off_new_4['tweet'][376] = "What an amazing seven-year journey it's been! #MyTwitterAnniversary [URL]."

<ipython-input-414-b0ee91e33c27>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][376] = "What an amazing seven-year journey it's been! #MyTwitterAnniversary [URL]."


In [415]:
test_tweet_off_new_4['tweet'][681] = "Can you even imagine? A hundred rounds? You've got to be joking, Goon! 🤣"

<ipython-input-415-073554772cc1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][681] = "Can you even imagine? A hundred rounds? You've got to be joking, Goon! 🤣"


In [416]:
test_tweet_off_new_4['tweet'][797]  = "To get folks to head out from your place, consider a humorous touch, like playfully acting like a vampire. Just ensure it's all in good fun and doesn't make anyone uneasy. Always approach situations with kindness and without being intimidating."

<ipython-input-416-2456e9ea89aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][797]  = "To get folks to head out from your place, consider a humorous touch, like playfully acting like a vampire. Just ensure it's all in good fun and doesn't make anyone uneasy. Always approach situations with kindness and without being intimidating."


In [417]:
test_tweet_off_new_4['tweet'][749] = "#Antifa intends to oppose the NVU's fascist views in #Amsterdam on September 22. [URL]"

<ipython-input-417-ab002ef90309>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][749] = "#Antifa intends to oppose the NVU's fascist views in #Amsterdam on September 22. [URL]"


In [418]:
test_tweet_off_new_4['tweet'][362]  = "#Share theleadersmindset • • • 📍Hey Leaders!🔥 ✔️Comment 'SET' if you're geared up for these actions. - 'Determine objectives, hold them close, tackle them with zeal, rejoice in your achievements... [URL]"

<ipython-input-418-6f0ae2a7a3e3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][362]  = "#Share theleadersmindset • • • 📍Hey Leaders!🔥 ✔️Comment 'SET' if you're geared up for these actions. - 'Determine objectives, hold them close, tackle them with zeal, rejoice in your achievements... [URL]"


In [419]:
test_tweet_off_new_4['tweet'][483] ="@USER @USER @USER Your meme seems to indicate that both Fascism and Antifa stood against liberals."

<ipython-input-419-9dfe6bbaebfc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][483] ="@USER @USER @USER Your meme seems to indicate that both Fascism and Antifa stood against liberals."


In [420]:
test_tweet_off_new_4['tweet'][313] ="@USER @USER @USER A lot of people perceive her as insincere within the Senate. If she represented me, I'd be disappointed by her actions during the SCOTUS hearings."

<ipython-input-420-4059ca66243c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][313] ="@USER @USER @USER A lot of people perceive her as insincere within the Senate. If she represented me, I'd be disappointed by her actions during the SCOTUS hearings."


In [421]:
test_tweet_off_new_4['tweet'][382] = "The circumstances involving #ChristineBlaseyFord have garnered notice. Some observers claim she's experiencing undue pressure from specific factions. They highlight alleged ties between Antifa, viewed by some as an extreme wing of the Democratic Party, and purported support from George Soros. The hashtag #VoteRedToSaveAmerica is associated with these discussions."

<ipython-input-421-08944d046359>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][382] = "The circumstances involving #ChristineBlaseyFord have garnered notice. Some observers claim she's experiencing undue pressure from specific factions. They highlight alleged ties between Antifa, viewed by some as an extreme wing of the Democratic Party, and purported support from George Soros. The hashtag #VoteRedToSaveAmerica is associated with these discussions."


In [422]:
test_tweet_off_new_4['tweet'][853] ="Both possess a displeasing flavor. [URL]."

<ipython-input-422-225074feafdd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_tweet_off_new_4['tweet'][853] ="Both possess a displeasing flavor. [URL]."


In [423]:
test_tweet_off_4 = test_tweet_off_new_4.loc[index_new3]

In [424]:

merged_df_4 = pd.concat([test_tweet_not_new, test_tweet_off_4], ignore_index=True)

# Shuffle the index
test_4_df = merged_df_4.sample(frac=1,random_state=42).reset_index(drop=True)

In [425]:
test_4_df.shape

(125, 2)

In [426]:
clean_tweets(test_4_df)

<ipython-input-10-58d0d99cd0a4>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
<ipython-input-10-58d0d99cd0a4>:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')


In [427]:
test_4_df.update(test_4_df[['tweet']].applymap('\'{}\''.format))

In [428]:

test_4_df["class_a_code"] = encoder.fit_transform(test_4_df["class_a"])

In [429]:
X_test_4 = test_4_df['tweet']
y_test_4 = test_4_df['class_a_code']

In [430]:
encoder = LabelEncoder()
train_labels = encoder.fit_transform(y_train)

test_labels_4 = encoder.transform(y_test_4)

In [431]:
test_encodings_4 = tokenizer(list(X_test_4), truncation=True, padding=True, max_length=256)

In [432]:
test_dataset_4 = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings_4),
    test_labels_4
)).batch(16)

In [433]:
# Get the model predictions
predictions_4 = tf.argmax(model.predict(test_dataset_4)[0], axis=-1)

8/8 [==============================] - 1s 101ms/step


In [434]:
# Decode the label numbers back into original labels
y_test_inverse_4 = encoder.inverse_transform(y_test_4)
predictions_inverse_4 = encoder.inverse_transform(predictions_4.numpy())

# Generate the classification report
report_4 = classification_report(y_test_inverse_4, predictions_inverse_4,
                               target_names=encoder.classes_.astype(str))
print(report_4)

              precision    recall  f1-score   support

           0       0.79      0.85      0.82       100
           1       0.12      0.08      0.10        25

    accuracy                           0.70       125
   macro avg       0.45      0.46      0.46       125
weighted avg       0.65      0.70      0.67       125

